In [1]:
import tensorflow as tf
import os
import collections
import numpy as np

# TODO(zongheng): read from argparse.
# TODO(zongheng): visualization via tensorboard.
# TODO(zongheng): print some prediction from test set

# Toy dataset.
TRAIN_PATH = "data/train-5rel-10numAttrs-26totalAttrs-100maxTblSize-100buckets-30.csv"
TEST_PATH = "data/test-5rel-10numAttrs-26totalAttrs-100maxTblSize-100buckets-30.csv"
NUM_FIELDS = 38
net = [64]
lr = 5e-4
batch_size = 1
num_steps = 7000

# 1000 examples in training.
# Min train loss < 0.6
TRAIN_PATH = "data/train-5rel-10numAttrs-17totalAttrs-100maxTblSize-100buckets-3000.csv"
TEST_PATH = "data/test-5rel-10numAttrs-17totalAttrs-100maxTblSize-100buckets-3000.csv"
NUM_FIELDS = 29
net = [64]
lr = 3e-3
batch_size = 128
num_steps = 5500


# With gaussian weights.
PATH = "5rel-10numAttrs-24totalAttrs-100maxTblSize-10buckets-300.csv"
NUM_FIELDS = 36
TRAIN_PATH = "data/train-{}".format(PATH)
TEST_PATH = "data/test-{}".format(PATH)
net = [128, 32, 64]
lr = 3e-3
lr = 1e-2
batch_size = 50
num_steps = 5500

# With gaussian weights.\
PATH = "5rel-10numAttrs-22totalAttrs-100maxTblSize-10buckets-3000.csv"
NUM_FIELDS = 34
TRAIN_PATH = "data/train-{}".format(PATH)
TEST_PATH = "data/test-{}".format(PATH)
net = [64,32]; lr = 1e-2
net = [64, 16, 64]; lr = 5e-3
net = [64]; lr = 1e-3
batch_size = 128
num_steps = 5500

/Users/zongheng/anaconda/envs/ray-0321/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
def DatasetFromPath(pathname):
    dataset = tf.data.TextLineDataset(pathname)

    def ParseLine(line):
        fields = tf.decode_csv(line, [[0.0]] * NUM_FIELDS)
        # Last number is label.
        return tf.reshape(tf.stack(fields[:-1]), [-1]), fields[-1]

    def MeanCenterBatch(feat, lab):
        feat_mean, feat_var = tf.nn.moments(feat, axes=[0])
        lab_mean, lab_var = tf.nn.moments(lab, axes=[0])
        feat = (feat - feat_mean) / (feat_var + 1e-4)
        lab = (lab - lab_mean) / (lab_var + 1e-4)
        return feat, lab
    
    dataset = dataset.map(ParseLine)
    # dataset = dataset.shuffle(2000).repeat().batch(batch_size)
#     dataset = dataset.shuffle_and_repeat(20000).batch(batch_size)
    dataset = dataset.shuffle(20000, reshuffle_each_iteration=True) \
        .batch(batch_size) \
        .repeat()
#         .map(MeanCenterBatch) \

    print(dataset.output_types)
    print(dataset.output_shapes)
    return dataset

# Train.
dataset = DatasetFromPath(TRAIN_PATH)
iterator = tf.data.Iterator.from_structure(
    dataset.output_types, dataset.output_shapes)
iterator_init = iterator.make_initializer(dataset)
# Test.
test_dataset = DatasetFromPath(TEST_PATH)
test_iterator_init = iterator.make_initializer(test_dataset)
# Shared.
feature_batch, label_batch = iterator.get_next()

    

# mean, var = tf.nn.moments(feature_batch, axes=[0])
# feature_batch = (feature_batch - mean) / tf.sqrt(var)
# mean, var = tf.nn.moments(label_batch, axes=[0])
# label_batch = (label_batch - mean) / var

(tf.float32, tf.float32)
(TensorShape([Dimension(None), Dimension(33)]), TensorShape([Dimension(None)]))
(tf.float32, tf.float32)
(TensorShape([Dimension(None), Dimension(33)]), TensorShape([Dimension(None)]))


In [3]:
# Graph definition.
for i in range(len(net)):
    dense = tf.layers.dense(
        dense if i > 0 else feature_batch,
        net[i],
        kernel_initializer=tf.truncated_normal_initializer,
#         kernel_initializer=tf.initializers.variance_scaling(),
        activation=tf.nn.relu,
#         activation=tf.nn.tanh,
    )
out = tf.layers.dense(
    dense,
    1,
        kernel_initializer=tf.truncated_normal_initializer,
#         kernel_initializer=tf.initializers.variance_scaling(),
)

loss = tf.reduce_mean(tf.square(out - label_batch))
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
global_step = tf.train.get_or_create_global_step()
train_step = optimizer.minimize(
    loss, global_step=global_step)

tf.summary.scalar('Train Loss', loss)

INFO:tensorflow:Summary name Train Loss is illegal; using Train_Loss instead.


<tf.Tensor 'Train_Loss:0' shape=() dtype=string>

In [4]:
# sess = tf.InteractiveSession()

In [ ]:
# Train.
sess_args = {
             'checkpoint_dir': os.getcwd(),
#              'save_checkpoint_secs': 60, 
             'save_summaries_secs': 60,
             'log_step_count_steps': 5000,
             'hooks': [tf.train.NanTensorHook(loss),
                       tf.train.StopAtStepHook(last_step=10000),
#                       tf.train.LoggingTensorHook([global_step, loss], every_n_iter=500)
                      ],
            }

loss_vals = collections.deque(maxlen=40)
i = 0
# with tf.train.MonitoredTrainingSession(**sess_args) as sess:
#     while not sess.should_stop():
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    running_test_loss = 0.0
    while True:
        sess.run(iterator_init)
        _, loss_val, global_step_val = sess.run([train_step, loss, global_step])
        
        if i % 100 == 0:
            sess.run(test_iterator_init)
            test_loss_val, actual_labels, predicted_labels = sess.run([loss, label_batch, out])

            loss_vals.append(test_loss_val)
            avg_test_loss = np.mean(loss_vals)
            print('global_step', global_step_val, ', loss', loss_val, ', avg test loss', avg_test_loss)
            if i % 1000 == 0:
                print(' true labels', actual_labels[:20])
                print(' pred labels', predicted_labels[:20].reshape(-1,))
        i += 1

global_step 1 , loss 57772.86 , avg test loss 57732.234
 true labels [ 2.  1.  1. 13.  8.  1. 14.  8. 10.  9.  8.  1.  1.  4.  1. 13.  1.  8.
 11. 13.]
 pred labels [185.3111   115.60453  408.39185  166.36569  124.35801  361.5647
 109.806526 216.58649  185.28848  320.46982  177.49396   96.40482
 226.3877   122.84504  138.63852  160.12936  118.6876   448.54288
 101.84559  131.57129 ]
global_step 101 , loss 681.41187 , avg test loss 29244.17
global_step 201 , loss 230.93488 , avg test loss 19611.63
global_step 301 , loss 238.6412 , avg test loss 14769.838
global_step 401 , loss 135.23615 , avg test loss 11843.418
global_step 501 , loss 160.80075 , avg test loss 9896.5205
global_step 601 , loss 222.72032 , avg test loss 8506.685
global_step 701 , loss 180.89676 , avg test loss 7467.824
global_step 801 , loss 239.87991 , avg test loss 6652.2207
global_step 901 , loss 211.84882 , avg test loss 5992.159
global_step 1001 , loss 53.047638 , avg test loss 5466.6055
 true labels [ 1.  1. 15.  1.